PPCA lavora con più variabili osservabili anzichè una sola

In [ ]:
import numpy as np
import pymc as pm
import matplotlib.pyplot as plt
import aesara.tensor as at
import arviz as az
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as metrics
from scipy import stats
import scipy

In [ ]:
scaler = StandardScaler()

In [ ]:
data = np.load('prova.npy', allow_pickle=True).item()

# extract data social anxiety

In [ ]:
import pandas as pd

In [ ]:
csv_ = '/Users/marcoghezzi/PycharmProjects/pythonProject/osfstorage-archive/behavior/LookAtMe_002.csv'
#csv_ = '/home/paolo/matteo/matteo/unimi/tesi_master/code/osfstorage-archive/behavior/LookAtMe_002.csv'
global_data = pd.read_csv(csv_, sep='\t')
y = np.array(list([int(d>2) for d in global_data['rating']]))
e_labels = y[:,np.newaxis]  # rating > 2

In [ ]:
N_e = e_labels.shape[0]
D_e = e_labels.shape[1]
print(N_e,D_e)

## test with new data

In [ ]:
import deepemogp.feature_extractor as feature_extractor
import deepemogp.signal.physio as physio
import deepemogp.signal.face as face
import deepemogp.signal.annotation as annotation
import deepemogp.datasets as datasets
import deepemogp.signal.behavior as behavior

# from: https://github.com/SheffieldML/GPy
#import GPy
# from: https://github.com/SheffieldML/PyDeepGP
#import deepgp

import pandas as pd
import numpy as np

show = False
# definition of the feature extractors to be used later
f1 = feature_extractor.FE('wavelet')
f2 = feature_extractor.FE('wavelet', window=(2, 1))
f3 = feature_extractor.FE('mean', window=(2, 1))

# definition of the physiological signals to be extracted
eda_ = physio.EDA(f2)
hr_ = physio.HR(f2)

pupil_ = behavior.PUPIL(f2)

# definition of the emotional annotation to be extracted
#va = annotation.VA('valence', f3)
#ar = annotation.VA('arousal', f3)
# extraction of the desired data from the dataset
d = datasets.FEAR(signals={hr_,pupil_}, subjects={'2_69'})

for s in d.signals:
    # preprocess ...
    #s.preprocess(show=show)

    # ... and extract features from each signal type
    s.feature_ext.extract_feat_without_preprocess(s, show=show)

hr_data = list(d.signals)[0].features
pupil_data = list(d.signals)[1].features

In [ ]:
hr_raw = list(d.signals)[0].raw[0]['data']

In [ ]:
d.signals

In [ ]:
hr_data

In [ ]:
pupil_data

## end test

prendo un'implementazione di sppca e la applico al nostro caso

different types of observable data

1) gaze
2) fisio
    2.1) heart rate variability
    2.2) eda phasic value
3) social anxiety
4) aspettativa del dolore

In [ ]:
pupil = pupil_data
hr = hr_data
#eda = data['eda']
'''social_anx = data['social anxiety'][:,np.newaxis]
pain_exp = data['pain expectation'][:,np.newaxis]'''


In [ ]:
print(pupil.shape)
print(hr.shape)

In [ ]:
N_pupil = pupil.shape[0]
D_pupil = pupil.shape[1]

N_hr = hr.shape[0]
D_hr = hr.shape[1]

#N_eda = eda.shape[0]
#D_eda = eda.shape[1]
K = 1

print(N_pupil,D_pupil)
print(N_hr,D_hr)

In [ ]:
with pm.Model() as sPPCA:
    #dati osservabili
    pupil_data = pm.MutableData("pupil_data", pupil.T)
    hr_data = pm.MutableData("hr_data", hr.T)
    #eda_data = pm.MutableData("eda_data", eda.T)
    e_data = pm.ConstantData("e_data", e_labels.T)

    #matrici pesi
    Wpupil = pm.Normal('Wpupil', mu=at.zeros([D_pupil, K]), sigma=2.0 * at.ones([D_pupil, K]), shape=[D_pupil, K])
    Whr = pm.Normal('Whr', mu=at.zeros([D_hr, K]), sigma=2.0 * at.ones([D_hr, K]), shape=[D_hr, K])
    #Weda = pm.Normal('Weda', mu=at.zeros([D_eda, K]), sigma=2.0 * at.ones([D_eda, K]), shape=[D_eda, K])

    #weight matrix for pain expectation.
    #check mu,sigma,shape
    Wexp = pm.Normal('W e', mu = at.zeros([D_e,K]), sigma=2.0 * at.ones([D_e, K]),shape=[D_e,K])

    #latent space
    c = pm.Normal('c', mu=at.zeros([160, K]), sigma=at.ones([160, K]), shape=[160, K])

    # dati dell'hrv interpretati come una gaussiana
    x_hr = pm.Normal('x_hr', mu=Whr.dot(c.T), sigma=at.ones([D_hr,N_hr]) ,shape=[D_hr, N_hr], observed=hr_data)

    # dati della dilatazione pupille interpretati come una gaussiana
    x_pupil = pm.Normal('x_pupil', mu=Wpupil.dot(c.T), sigma=at.ones([D_pupil, N_pupil]), shape=[D_pupil, N_pupil], observed=pupil_data)

    # pain expectation. ciò che dovremmo inferire dato c
    # due strade: binary o multiclass (1-4)
    # p = probability of success?
    pain_expectation = pm.Bernoulli('pain_exp' , p=pm.math.sigmoid(Wexp.dot(c.T)) , shape =[D_e, N_e], observed=e_data)


    #x_hr = pm.Bernoulli('x_hr', p=pm.math.sigmoid(Whr.dot(c.T)), shape=[D_hr, N_hr], observed=hr_data)
    #x_eda = pm.Bernoulli('x_eda', p=pm.math.sigmoid(Weda.dot(c.T)), shape=[D_eda, N_eda], observed=eda_data)

In [ ]:
gv = pm.model_to_graphviz(sPPCA)
gv.view()

In [ ]:
sPPCA.free_RVs

In [ ]:
with sPPCA:
    trace = pm.sample(1000,init='advi+adapt_diag',chains=1,progressbar=True)

In [ ]:
#az.plot_posterior(mean_field.sample(1000), color="LightSeaGreen")
az.plot_trace(trace)
plt.show()

In [ ]:
with sPPCA:
    # update values of predictors:
    pm.set_data({"gaze_data": pupil})
    # use the updated values and predict outcomes and probabilities:
    posterior_predictive = pm.sample_posterior_predictive(
        trace, random_seed=123)
hr_pred = posterior_predictive.posterior_predictive["x_hr"]

In [ ]:
pupil

In [ ]:
print(hr_pred[0])

In [ ]:
print(hr_pred[0].shape)

In [ ]:
print(float(np.mean(hr_pred[0])))

median abs deviation prb is not the optimal solution

In [ ]:
hr_pred_mean = (stats.median_abs_deviation(hr_pred[0]))[0]
hr_pred_mean

In [ ]:
hr[0]

In [ ]:
#hr_pred_mode= pd.Series([0.006,0.01,0.02,0.003])

hr and hr_pred_mode devono avere same shape

In [ ]:
train_accuracy_hr = (np.subtract(hr[0], hr_pred_mean))
print('\n\t avg distance hr: \n' + str(train_accuracy_hr))
#print(' ')

In [ ]:
with sPPCA:
    # update values of predictors:
    pm.set_data({"hr_data": hr})
    # use the updated values and predict outcomes and probabilities:
    posterior_predictive = pm.sample_posterior_predictive(
        trace, random_seed=123)
    gaze_pred = posterior_predictive.posterior_predictive["x_gaze"]

In [ ]:
gaze_pred

In [ ]:
gaze_pred_mean = np.mean(gaze_pred, axis=0).T.reshape(-1)
gaze_true = pupil.reshape(-1)
reconstructed_r2 = metrics.r2_score(gaze_true, gaze_pred_mean)

print('\n\tReconstructed X R2 score: ' + str(reconstructed_r2))

In [ ]:
import deepemogp.feature_extractor as feature_extractor

In [ ]:
f = feature_extractor.FE('wavelet', window=(8,6), params={'w_mother':'db3','w_maxlev':1})

In [ ]:
x = [3, 7, 1, 1, -2, 5, 4, 6]

In [ ]:
f.apply(x)